In [1]:
import json
from textwrap import dedent

import httpx
from agno.agent import Agent
from agno.models.openai import OpenAIChat

In [2]:
def get_top_hackernews_stories(num_stories: int = 10) -> str:
    """Use this function to get top stories from Hacker News.

    Args:
        num_stories (int): Number of stories to return. Defaults to 10.

    Returns:
        str: JSON string of top stories.
    """

    # Fetch top story IDs
    response = httpx.get("https://hacker-news.firebaseio.com/v0/topstories.json")
    story_ids = response.json()

    # Fetch story details
    stories = []
    for story_id in story_ids[:num_stories]:
        story_response = httpx.get(
            f"https://hacker-news.firebaseio.com/v0/item/{story_id}.json"
        )
        story = story_response.json()
        if "text" in story:
            story.pop("text", None)
        stories.append(story)
    return json.dumps(stories)

In [5]:
agent = Agent(
    model=OpenAIChat(id="gpt-4o-mini"),
    instructions=dedent("""\
        Você é um repórter do Hacker News com uma paixão por tudo que é tecnologia! 🤖
        Pense em si mesmo como uma mistura entre um insider do Vale do Silício e um jornalista de tecnologia.

        Seu guia de estilo:
        - Comece com uma manchete chamativa usando emojis
        - Apresente as histórias do Hacker News com entusiasmo e atitude voltada para a tecnologia
        - Mantenha suas respostas concisas, mas informativas
        - Use referências da indústria de tecnologia e jargões de startups quando apropriado
        - Termine com uma assinatura temática de tecnologia, como 'De volta ao terminal!' ou 'Pushing to production!'

        Lembre-se de analisar as histórias do HN de forma completa, mantendo o entusiasmo tecnológico alto!\
    """),
    tools=[get_top_hackernews_stories],
    show_tool_calls=True,
    markdown=True,
)

In [7]:
agent.print_response(
    "Resuma as 5 principais histórias do Hacker News, destacando inovações tecnológicas e tendências.", 
    stream=True
)

Output()